In [274]:
#!usr/bin/env python3
import os
import shutil
from zipfile import ZipFile 
import pandas as pd
import numpy as np
import dateutil #https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
from datetime import datetime
import glob
import time
import gc  #garbage collection to free up memory

# Hide warning messages in notebook
import warnings
warnings.filterwarnings('ignore')


In [275]:
#set some static parameters
debug_mode = 'y'
csv_header_ind = 'True'
cur_dir = os.getcwd()


## Load Data

In [276]:
if debug_mode == 'y':
    data_year = 9999
    print(data_year)
else:
    for i in range(2013,2014):
        data_year = i
        print(data_year)        


9999


In [277]:
# print start timestamp 
execStartDateTime = datetime.now()
print(execStartDateTime)


2020-06-21 16:41:36.344167


In [278]:
raw_dir = os.path.join(cur_dir,'citibike_files','raw', str(data_year))

In [279]:
path = raw_dir

all_files = sorted(glob.glob(os.path.join(path, "*citibike*")))

all_df = []

#parse_dates=['starttime','stoptime'],

for f in all_files:
    try: 
        print('try:' + f)
        df = pd.read_csv(f, sep=',',header = 1,skiprows = 1,\
        names=['tripduration','starttime','stoptime','start station id','start station name'\
        ,'start station latitude','start station longitude','end station id','end station name'\
        ,'end station latitude','end station longitude','bikeid','usertype','birth year','gender'])
        print(len(df.index))
        df['a_file'] = f.split('/')[-1]    
        all_df.append(df)
        citibike_df = pd.concat(all_df, ignore_index=True, sort=True)
    except:
         print('except:' + f)

try:/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/raw/9999/201512-citibike-tripdata.zip
804123


## Pre-processing: Preview data & datatype inspection

In [280]:
citibike_df= pd.DataFrame(citibike_df)

In [281]:
list(citibike_df.columns) 

['a_file',
 'bikeid',
 'birth year',
 'end station id',
 'end station latitude',
 'end station longitude',
 'end station name',
 'gender',
 'start station id',
 'start station latitude',
 'start station longitude',
 'start station name',
 'starttime',
 'stoptime',
 'tripduration',
 'usertype']

In [282]:
citibike_df.dtypes

a_file                      object
bikeid                       int64
birth year                 float64
end station id               int64
end station latitude       float64
end station longitude      float64
end station name            object
gender                       int64
start station id             int64
start station latitude     float64
start station longitude    float64
start station name          object
starttime                   object
stoptime                    object
tripduration                 int64
usertype                    object
dtype: object

In [283]:
# set text columns as categories
for col in ['gender', 'usertype', 'start station name', 'end station name']:
    citibike_df[col] = citibike_df[col].astype('category')

In [284]:
# set datatypes for numeric columns
citibike_df['start station id'] = citibike_df['start station id'].astype(str).astype(float).astype(int)
citibike_df['start station latitude'] = citibike_df['start station latitude'].astype(float)
citibike_df['start station latitude'] = citibike_df['start station latitude'].round(decimals=3)
citibike_df['start station longitude'] = citibike_df['start station longitude'].astype(float)
citibike_df['start station longitude'] = citibike_df['start station longitude'].round(decimals=3)
citibike_df = citibike_df.dropna(subset=['end station id'])
citibike_df['end station id'] = citibike_df['end station id'].astype(str).astype(float).astype(int)
citibike_df['end station latitude'] = citibike_df['end station latitude'].astype(float)
citibike_df['end station latitude'] = citibike_df['end station latitude'].round(decimals=3)
citibike_df['end station longitude'] = citibike_df['end station longitude'].astype(float)
citibike_df['end station longitude'] = citibike_df['end station longitude'].round(decimals=3)

In [285]:
citibike_df.dtypes

a_file                       object
bikeid                        int64
birth year                  float64
end station id                int64
end station latitude        float64
end station longitude       float64
end station name           category
gender                     category
start station id              int64
start station latitude      float64
start station longitude     float64
start station name         category
starttime                    object
stoptime                     object
tripduration                  int64
usertype                   category
dtype: object

In [286]:
citibike_df.head()

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype
0,201512-citibike-tripdata.zip,18797,1966.0,358,40.733,-74.007,Christopher St & Greenwich St,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 07:44:49,12/1/2015 08:02:33,1063,Subscriber
1,201512-citibike-tripdata.zip,14625,1985.0,505,40.749,-73.988,6 Ave & W 33 St,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:02:29,12/1/2015 08:20:24,1075,Subscriber
2,201512-citibike-tripdata.zip,21238,1968.0,525,40.756,-74.002,W 34 St & 11 Ave,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:06:37,12/1/2015 08:11:30,293,Subscriber
3,201512-citibike-tripdata.zip,19518,1960.0,484,40.755,-73.980,W 44 St & 5 Ave,2,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:07:57,12/1/2015 08:21:30,812,Subscriber
4,201512-citibike-tripdata.zip,22307,1980.0,520,40.760,-73.976,W 52 St & 5 Ave,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:08:53,12/1/2015 08:18:05,551,Subscriber


In [287]:
citibike_df['birth year'].value_counts()

1984.0    29441
1985.0    29271
1983.0    28169
1986.0    27495
1987.0    26811
          ...  
1926.0       13
1918.0        8
1934.0        5
1917.0        2
1907.0        2
Name: birth year, Length: 77, dtype: int64

In [288]:
# Using try block here since data files were not consistent over time
try:
    if pd.api.types.is_string_dtype:
        citibike_df['birth year'] = citibike_df['birth year'].replace({"\\N":2020})
except:
    print("skip")

skip


In [289]:
citibike_df['birth year'].fillna(2020,inplace=True)

In [290]:
# Now that all fields are prepped drop nans in dataframe.  This is slow.
citibike_df.dropna(inplace=True)

In [291]:
# Set birth year datatype the nans dropped
citibike_df['birth year'] = citibike_df['birth year'].astype(str).astype(float).astype(int)

In [292]:
citibike_df.isnull().sum(axis=0)

a_file                     0
bikeid                     0
birth year                 0
end station id             0
end station latitude       0
end station longitude      0
end station name           0
gender                     0
start station id           0
start station latitude     0
start station longitude    0
start station name         0
starttime                  0
stoptime                   0
tripduration               0
usertype                   0
dtype: int64

In [293]:
#stamp the output files yearmonth to track the source of the data
citibike_df['yearmonth'] =  citibike_df['a_file'].str[:6].astype(int)
citibike_df['tripduration'] = citibike_df['tripduration']/60

## Analyze by date and starthour

In [294]:
citibike_df.dtypes

a_file                       object
bikeid                        int64
birth year                    int64
end station id                int64
end station latitude        float64
end station longitude       float64
end station name           category
gender                     category
start station id              int64
start station latitude      float64
start station longitude     float64
start station name         category
starttime                    object
stoptime                     object
tripduration                float64
usertype                   category
yearmonth                     int64
dtype: object

In [295]:
citibike_df[['begindate','begintime']] = citibike_df.starttime.str.split(expand=True) 

In [296]:
# Possible optimzation:  https://stackoverflow.com/questions/50744369/how-to-speed-up-pandas-string-function
# %timeit [x.split('~', 1)[0] for x in df['facility']]
# def splittime(x):
#     test = [x.split(' ', 1)[0] for x in citibike_df['starttime']]
#     return x.map(test)
# citibike_df['test2'] = splittime(citibike_df['starttime'])
# TypeError: list indices must be integers or slices, not str   

In [297]:
# https://github.com/pandas-dev/pandas/issues/11665
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [298]:
citibike_df['startdate'] = lookup(citibike_df['begindate'])

In [299]:
citibike_df.head()

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,start station longitude,start station name,starttime,stoptime,tripduration,usertype,yearmonth,begindate,begintime,startdate
0,201512-citibike-tripdata.zip,18797,1966,358,40.733,-74.007,Christopher St & Greenwich St,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 07:44:49,12/1/2015 08:02:33,17.716667,Subscriber,201512,12/1/2015,07:44:49,2015-12-01
1,201512-citibike-tripdata.zip,14625,1985,505,40.749,-73.988,6 Ave & W 33 St,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:02:29,12/1/2015 08:20:24,17.916667,Subscriber,201512,12/1/2015,08:02:29,2015-12-01
2,201512-citibike-tripdata.zip,21238,1968,525,40.756,-74.002,W 34 St & 11 Ave,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:06:37,12/1/2015 08:11:30,4.883333,Subscriber,201512,12/1/2015,08:06:37,2015-12-01
3,201512-citibike-tripdata.zip,19518,1960,484,40.755,-73.980,W 44 St & 5 Ave,2,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:07:57,12/1/2015 08:21:30,13.533333,Subscriber,201512,12/1/2015,08:07:57,2015-12-01
4,201512-citibike-tripdata.zip,22307,1980,520,40.760,-73.976,W 52 St & 5 Ave,1,72,40.767,-73.994,W 52 St & 11 Ave,12/1/2015 08:08:53,12/1/2015 08:18:05,9.183333,Subscriber,201512,12/1/2015,08:08:53,2015-12-01


In [300]:
citibike_df['starthour'] = citibike_df['begintime'].str.slice(0, 2)

In [301]:
daily_df = citibike_df.groupby(['startdate']).tripduration.agg(['count','sum']).reset_index().set_index(['startdate'])
daily_df.sort_index(axis = 0) 
daily_df

,count,sum
startdate,,
2015-12-01,18397,236970.933333
2015-12-02,23782,321226.666667
2015-12-03,33917,446049.800000
2015-12-04,34737,505553.833333
2015-12-05,26358,559490.150000
2015-12-06,24943,490887.333333
2015-12-07,34139,481600.500000
2015-12-08,35614,487627.500000
2015-12-09,35306,513159.616667


In [302]:
citibike_daily_bike_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_trips_daily.csv')

In [303]:
if debug_mode == 'n':
    if not os.path.isfile(citibike_daily_bike_csv):
       daily_df.to_csv(citibike_daily_bike_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       daily_df.to_csv(citibike_daily_bike_csv, mode='a', header=False, index=False)

In [304]:
# Extend analysis tostart hour
hourly_df = citibike_df.groupby(['startdate','starthour']).tripduration.agg(['count','sum']).reset_index()
hourly_df.set_index('startdate', inplace = True) 
hourly_df.sort_index(axis = 0) 
hourly_df.head()

,starthour,count,sum
startdate,,,
2015-12-01,00,136,1832.900000
2015-12-01,01,54,875.683333
2015-12-01,02,27,716.350000
2015-12-01,03,15,247.166667
2015-12-01,04,19,156.766667


In [305]:
citibike_hourly_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_trips_hourly.csv')

In [306]:
if debug_mode == 'n':
    if not os.path.isfile(citibike_hourly_csv):
       hourly_df.to_csv(citibike_hourly_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       hourly_df.to_csv(citibike_hourly_csv, mode='a', header=False, index=False)

## Analyze customer data

In [307]:
citibike_df['gender'].value_counts()

1    568229
2    166368
0     69526
Name: gender, dtype: int64

In [308]:
citibike_df['birth year'].value_counts()

2020    67877
1984    29441
1985    29271
1983    28169
1986    27495
        ...  
1926       13
1918        8
1934        5
1917        2
1907        2
Name: birth year, Length: 78, dtype: int64

In [309]:
currentYear = datetime.now().year

In [310]:
citibike_df['rider age'] = currentYear - citibike_df['birth year']
citibike_df

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,...,starttime,stoptime,tripduration,usertype,yearmonth,begindate,begintime,startdate,starthour,rider age
0,201512-citibike-tripdata.zip,18797,1966,358,40.733,-74.007,Christopher St & Greenwich St,1,72,40.767,...,12/1/2015 07:44:49,12/1/2015 08:02:33,17.716667,Subscriber,201512,12/1/2015,07:44:49,2015-12-01,07,54
1,201512-citibike-tripdata.zip,14625,1985,505,40.749,-73.988,6 Ave & W 33 St,1,72,40.767,...,12/1/2015 08:02:29,12/1/2015 08:20:24,17.916667,Subscriber,201512,12/1/2015,08:02:29,2015-12-01,08,35
2,201512-citibike-tripdata.zip,21238,1968,525,40.756,-74.002,W 34 St & 11 Ave,1,72,40.767,...,12/1/2015 08:06:37,12/1/2015 08:11:30,4.883333,Subscriber,201512,12/1/2015,08:06:37,2015-12-01,08,52
3,201512-citibike-tripdata.zip,19518,1960,484,40.755,-73.980,W 44 St & 5 Ave,2,72,40.767,...,12/1/2015 08:07:57,12/1/2015 08:21:30,13.533333,Subscriber,201512,12/1/2015,08:07:57,2015-12-01,08,60
4,201512-citibike-tripdata.zip,22307,1980,520,40.760,-73.976,W 52 St & 5 Ave,1,72,40.767,...,12/1/2015 08:08:53,12/1/2015 08:18:05,9.183333,Subscriber,201512,12/1/2015,08:08:53,2015-12-01,08,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804118,201512-citibike-tripdata.zip,22282,1957,261,40.695,-73.984,Johnson St & Gold St,1,3242,40.691,...,12/31/2015 16:50:17,12/31/2015 16:58:46,8.483333,Subscriber,201512,12/31/2015,16:50:17,2015-12-31,16,63
804119,201512-citibike-tripdata.zip,22366,1978,258,40.689,-73.969,DeKalb Ave & Vanderbilt Ave,1,3242,40.691,...,12/31/2015 17:17:52,12/31/2015 17:27:43,9.850000,Subscriber,201512,12/31/2015,17:17:52,2015-12-31,17,42
804120,201512-citibike-tripdata.zip,20663,1959,157,40.691,-73.996,Henry St & Atlantic Ave,1,3242,40.691,...,12/31/2015 17:56:22,12/31/2015 17:59:39,3.283333,Subscriber,201512,12/31/2015,17:56:22,2015-12-31,17,61
804121,201512-citibike-tripdata.zip,19782,1987,467,40.683,-73.979,Dean St & 4 Ave,1,3242,40.691,...,12/31/2015 21:20:38,12/31/2015 21:26:50,6.200000,Subscriber,201512,12/31/2015,21:20:38,2015-12-31,21,33


In [311]:
bins = [-1,1,18,25,45,65,100,1000]
citibike_df['age bracket'] = pd.cut(citibike_df['rider age'],bins)
citibike_df

,a_file,bikeid,birth year,end station id,end station latitude,end station longitude,end station name,gender,start station id,start station latitude,...,stoptime,tripduration,usertype,yearmonth,begindate,begintime,startdate,starthour,rider age,age bracket
0,201512-citibike-tripdata.zip,18797,1966,358,40.733,-74.007,Christopher St & Greenwich St,1,72,40.767,...,12/1/2015 08:02:33,17.716667,Subscriber,201512,12/1/2015,07:44:49,2015-12-01,07,54,"(45, 65]"
1,201512-citibike-tripdata.zip,14625,1985,505,40.749,-73.988,6 Ave & W 33 St,1,72,40.767,...,12/1/2015 08:20:24,17.916667,Subscriber,201512,12/1/2015,08:02:29,2015-12-01,08,35,"(25, 45]"
2,201512-citibike-tripdata.zip,21238,1968,525,40.756,-74.002,W 34 St & 11 Ave,1,72,40.767,...,12/1/2015 08:11:30,4.883333,Subscriber,201512,12/1/2015,08:06:37,2015-12-01,08,52,"(45, 65]"
3,201512-citibike-tripdata.zip,19518,1960,484,40.755,-73.980,W 44 St & 5 Ave,2,72,40.767,...,12/1/2015 08:21:30,13.533333,Subscriber,201512,12/1/2015,08:07:57,2015-12-01,08,60,"(45, 65]"
4,201512-citibike-tripdata.zip,22307,1980,520,40.760,-73.976,W 52 St & 5 Ave,1,72,40.767,...,12/1/2015 08:18:05,9.183333,Subscriber,201512,12/1/2015,08:08:53,2015-12-01,08,40,"(25, 45]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804118,201512-citibike-tripdata.zip,22282,1957,261,40.695,-73.984,Johnson St & Gold St,1,3242,40.691,...,12/31/2015 16:58:46,8.483333,Subscriber,201512,12/31/2015,16:50:17,2015-12-31,16,63,"(45, 65]"
804119,201512-citibike-tripdata.zip,22366,1978,258,40.689,-73.969,DeKalb Ave & Vanderbilt Ave,1,3242,40.691,...,12/31/2015 17:27:43,9.850000,Subscriber,201512,12/31/2015,17:17:52,2015-12-31,17,42,"(25, 45]"
804120,201512-citibike-tripdata.zip,20663,1959,157,40.691,-73.996,Henry St & Atlantic Ave,1,3242,40.691,...,12/31/2015 17:59:39,3.283333,Subscriber,201512,12/31/2015,17:56:22,2015-12-31,17,61,"(45, 65]"
804121,201512-citibike-tripdata.zip,19782,1987,467,40.683,-73.979,Dean St & 4 Ave,1,3242,40.691,...,12/31/2015 21:26:50,6.200000,Subscriber,201512,12/31/2015,21:20:38,2015-12-31,21,33,"(25, 45]"


In [312]:
customer_df = citibike_df.groupby(['startdate','gender','age bracket','usertype']).tripduration.agg(['count']).reset_index()
customer_df

,startdate,gender,age bracket,usertype,count
0,2015-12-01,0,"(-1, 1]",Customer,441
1,2015-12-01,0,"(-1, 1]",Subscriber,0
2,2015-12-01,0,"(1, 18]",Customer,0
3,2015-12-01,0,"(1, 18]",Subscriber,0
4,2015-12-01,0,"(18, 25]",Customer,0
...,...,...,...,...,...
1297,2015-12-31,2,"(45, 65]",Subscriber,1313
1298,2015-12-31,2,"(65, 100]",Customer,0
1299,2015-12-31,2,"(65, 100]",Subscriber,185
1300,2015-12-31,2,"(100, 1000]",Customer,0


In [313]:
citibike_customer_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_customers.csv')

In [314]:
if debug_mode == 'n':
    #https://stackoverflow.com/questions/30991541/pandas-write-csv-append-vs-write
    if not os.path.isfile(citibike_customer_csv):
       customer_df.to_csv(citibike_customer_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       customer_df.to_csv(citibike_customer_csv, mode='a', header=False, index=False)


## Analyze bike stations

In [315]:
start_stations_df = citibike_df.drop_duplicates(subset=["start station id", "start station latitude","start station longitude","start station name"])
start_stations_df = start_stations_df[["start station id", "start station latitude","start station longitude","start station name"]]
start_stations_df = pd.DataFrame(start_stations_df)
start_stations_df.columns = ["station id", "station latitude","station longitude","station name"]
start_stations_df

,station id,station latitude,station longitude,station name
0,72,40.767,-73.994,W 52 St & 11 Ave
1646,79,40.719,-74.007,Franklin St & W Broadway
3225,82,40.711,-74.000,St James Pl & Pearl St
3938,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
4655,116,40.742,-74.001,W 17 St & 8 Ave
...,...,...,...,...
801403,3236,40.759,-73.994,PABT Valet
803500,3237,40.754,-73.943,21 St & 41 Ave
803622,3238,40.774,-73.954,E 80 St & 2 Ave
804095,3241,40.686,-73.945,Monroe St & Tompkins Ave


In [316]:
end_stations_df = citibike_df.drop_duplicates(subset=["end station id", "end station latitude","end station longitude","end station name"])
end_stations_df = end_stations_df[["end station id", "end station latitude","end station longitude","end station name"]]
end_stations_df = pd.DataFrame(end_stations_df)
end_stations_df.columns = ["station id", "station latitude","station longitude","station name"]
end_stations_df

,station id,station latitude,station longitude,station name
0,358,40.733,-74.007,Christopher St & Greenwich St
1,505,40.749,-73.988,6 Ave & W 33 St
2,525,40.756,-74.002,W 34 St & 11 Ave
3,484,40.755,-73.980,W 44 St & 5 Ave
4,520,40.760,-73.976,W 52 St & 5 Ave
...,...,...,...,...
127068,3017,40.751,-73.997,NYCBS Depot - FAR
151789,3128,40.751,-73.946,21 St & 43 Ave
152456,3059,40.693,-73.940,Pulaski St & Marcus Garvey Blvd
578241,3187,40.721,-74.038,Warren St


In [317]:
# distinct_stations_df = start_stations_df.append(end_stations_df)
# distinct_stations_df = distinct_stations_df.drop_duplicates(subset=["station id", "station latitude","station longitude","station name"])
# distinct_stations_df = distinct_stations_df.set_index('station id', inplace = True)
# #distinct_stations_df.sort_index(axis = 0) 
# distinct_stations_df

In [318]:
distinct_stations_df = pd.DataFrame(start_stations_df.append(end_stations_df))
distinct_stations_df

,station id,station latitude,station longitude,station name
0,72,40.767,-73.994,W 52 St & 11 Ave
1646,79,40.719,-74.007,Franklin St & W Broadway
3225,82,40.711,-74.000,St James Pl & Pearl St
3938,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
4655,116,40.742,-74.001,W 17 St & 8 Ave
...,...,...,...,...
127068,3017,40.751,-73.997,NYCBS Depot - FAR
151789,3128,40.751,-73.946,21 St & 43 Ave
152456,3059,40.693,-73.940,Pulaski St & Marcus Garvey Blvd
578241,3187,40.721,-74.038,Warren St


In [319]:
distinct_stations_df = distinct_stations_df.drop_duplicates(subset=["station id", "station latitude","station longitude","station name"])
distinct_stations_df

,station id,station latitude,station longitude,station name
0,72,40.767,-73.994,W 52 St & 11 Ave
1646,79,40.719,-74.007,Franklin St & W Broadway
3225,82,40.711,-74.000,St James Pl & Pearl St
3938,83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
4655,116,40.742,-74.001,W 17 St & 8 Ave
...,...,...,...,...
58658,3219,40.729,-73.977,NYCBS Depot - STY
65355,255,40.647,-74.017,NYCBS Depot - SSP
127068,3017,40.751,-73.997,NYCBS Depot - FAR
578241,3187,40.721,-74.038,Warren St


In [320]:
distinct_stations_df.set_index('station id', inplace = True)
distinct_stations_df.head()

,station latitude,station longitude,station name
station id,,,
72,40.767,-73.994,W 52 St & 11 Ave
79,40.719,-74.007,Franklin St & W Broadway
82,40.711,-74.000,St James Pl & Pearl St
83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
116,40.742,-74.001,W 17 St & 8 Ave


In [321]:
distinct_stations_df.sort_index(axis = 0) 
distinct_stations_df

,station latitude,station longitude,station name
station id,,,
72,40.767,-73.994,W 52 St & 11 Ave
79,40.719,-74.007,Franklin St & W Broadway
82,40.711,-74.000,St James Pl & Pearl St
83,40.684,-73.976,Atlantic Ave & Fort Greene Pl
116,40.742,-74.001,W 17 St & 8 Ave
...,...,...,...
3219,40.729,-73.977,NYCBS Depot - STY
255,40.647,-74.017,NYCBS Depot - SSP
3017,40.751,-73.997,NYCBS Depot - FAR


In [322]:
distinct_stations_df.isnull().sum()


station latitude     0
station longitude    0
station name         0
dtype: int64

In [323]:
citibike_distinct_station_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_distinct_stations.csv')

In [324]:
if debug_mode == 'n':
    if not os.path.isfile(citibike_distinct_station_csv):
       distinct_stations_df.to_csv(citibike_distinct_station_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       distinct_stations_df.to_csv(citibike_distinct_station_csv, mode='a', header=False, index=False)

In [325]:
start_station_trips_df = citibike_df.groupby(['startdate','start station id']).tripduration.agg(['count']).reset_index()
start_station_trips_df = start_station_trips_df.set_index(['startdate'])
start_station_trips_df

,start station id,count
startdate,,
2015-12-01,72,45
2015-12-01,79,40
2015-12-01,82,8
2015-12-01,83,9
2015-12-01,116,83
...,...,...
2015-12-31,3236,40
2015-12-31,3237,10
2015-12-31,3238,20


In [326]:
start_station_trips_df.isnull().sum()

start station id    0
count               0
dtype: int64

In [327]:
citibike_start_station_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_start_stations.csv')

In [328]:
if debug_mode == 'n':
    if not os.path.isfile(citibike_start_station_csv):
       start_station_trips_df.to_csv(citibike_start_station_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       start_station_trips_df.to_csv(citibike_start_station_csv, mode='a', header=False, index=False)

## Analyze bike equipment

In [329]:
bike_equipment_df = citibike_df.groupby(['bikeid']).tripduration.agg(['count','sum']).reset_index()
bike_equipment_df = bike_equipment_df.set_index('bikeid')
bike_equipment_df = pd.DataFrame(bike_equipment_df)

In [330]:
bike_date_df = citibike_df.groupby(['bikeid']).startdate.agg(['min','max']).reset_index()
bike_date_df = bike_date_df.set_index(['bikeid'])
bike_date_df = pd.DataFrame(bike_date_df)

In [331]:
bike_merged_df = pd.merge(bike_date_df, bike_equipment_df, left_index=True, right_index=True)
bike_merged_df = pd.DataFrame(bike_merged_df)

list

In [332]:
citibike_bike_equipment_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_bikes.csv')

In [333]:
if debug_mode == 'n':
    if not os.path.isfile(citibike_bike_equipment_csv):
       bike_merged_df.to_csv(citibike_bike_equipment_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       bike_merged_df.to_csv(citibike_bike_equipment_csv, mode='a', header=False, index=False)

## Cleanup memory for next run

In [334]:
del [[citibike_df]]
del [[start_stations_df, end_stations_df, distinct_stations_df]]
del [[bike_equipment_df, bike_date_df, bike_merged_df]]
gc.collect()
citibike_df = []
customer_df = []
distinct_stations_df = []
start_stations_df = []
end_stations_df = []
bike_equipment_df = []
bike_date_df = []
bike_merged_df = []

In [335]:
# Post-Processing:  Data is processed by year.  Final aggregagtion needed to poroduce distinct stations for all years.
citibike_distinct_station_final_df = pd.read_csv(os.path.join(cur_dir, 'citibike_files', 'cleansed', 'citibike_distinct_stations.csv'))
citibike_distinct_station_final_df = pd.DataFrame(citibike_distinct_station_final_df)
citibike_distinct_station_final_df = citibike_distinct_station_final_df.drop_duplicates(subset=["station id", "station latitude","station longitude","station name"],keep='first')
citibike_distinct_station_final_df = citibike_distinct_station_final_df[["station id", "station latitude","station longitude","station name"]]
citibike_distinct_station_final_df.columns = ["station id", "station latitude","station longitude","station name"]
citibike_distinct_station_final_df.set_index("station id", inplace=True)
citibike_distinct_station_final_df.sort_values("station id", axis = 0, ascending = True, inplace = True, na_position ='first')
citibike_distinct_station_final_df
citibike_distinct_station_final_csv = os.path.join(cur_dir,'citibike_files','cleansed','citibike_distinct_stations_final.csv')
if debug_mode == 'n':
    if not os.path.isfile(citibike_distinct_station_final_csv):
       citibike_distinct_station_final_df.to_csv(citibike_distinct_station_final_csv, header='column_names', index=False)
    else: # else it exists so append without writing the header
       citibike_distinct_station_final_df.to_csv(citibike_distinct_station_final_csv, mode='a', header=False, index=False)

FileNotFoundError: [Errno 2] File /Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/cleansed/citibike_distinct_stations.csv does not exist: '/Users/Werd/boot_camp/gitlib/tableau-citibike/citibike_files/cleansed/citibike_distinct_stations.csv'

In [336]:
# print end 
print(data_year)
execEndDateTime = datetime.now()
print(execStartDateTime)
print(execEndDateTime)

9999
2020-06-21 16:41:36.344167
2020-06-21 16:41:52.884236
